# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [1]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [2]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [3]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [4]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00 -1.110e-16  1.110e-16  1.110e-16 -5.551e-17]
 [ 0.000e+00  0.000e+00  3.331e-16 -2.220e-16 -5.551e-17  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -1.110e-16 -1.665e-16  0.000e+00]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [5]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [6]:
np.linalg.matrix_rank(a1)

6

In [7]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in matmul
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in matmul
/srv/conda/envs/notebook/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [8]:
def ifLU(A):
    from numpy.linalg import det
    for i in range(A.shape[0]):
        if det(A[:i,:i]) == 0:
            return False
    return True

print('LU decomposition is available for matrix a:', ifLU(a))
print('LU decomposition is available for matrix a1:', ifLU(a1))

LU decomposition is available for matrix a: True
LU decomposition is available for matrix a1: False


### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [11]:
import numpy as np

def pivot(A):
    M = A.copy()
    N = A.shape[0]
    P = np.eye(N)
    
    for i in range(N):
        temp = np.eye(N)
        row = np.argmax(np.abs(M[:,i][i:])) + i
        temp[[i, row]] = temp[[row, i]]
        P = temp @ P
        M = temp @ M

    return P

def LUP_decomposition(A):
    N = A.shape[0]    
    P = np.eye(N)
    U = A.copy()
    L = np.eye(N)
    
    for j in range(N - 1):
        lam = np.eye(N)
        gamma = U[j + 1:, j] / U[j, j]
        lam[j + 1:, j] = -gamma
        U = lam @ U
        pMatrix = pivot(U).T
        U = U @ pMatrix
        lam[j + 1:, j] = gamma
        L = L @ lam
        P = P @ pMatrix
    return L, U, P.T

np.set_printoptions(suppress=True, linewidth=100)
print('----------------------------------------------------------------\n LUP for matrix a:')
L, U, P = LUP_decomposition(a)
print('Matrix L:')
print(L)
print('Matrix U:')
print(U)
print('Permutation matrix:')
print(P)
print('Initial matrix:')
print(a)
print('LUP:')
print(L @ U @ P)

print('----------------------------------------------------------------\n LUP for matrix a1:')
L, U, P = LUP_decomposition(a1)
print('Matrix L:')
print(L)
print('Matrix U:')
print(U)
print('Permutation matrix:')
print(P)
print('Initial matrix:')
print(a1)
print('LUP:')
print(L @ U @ P)

----------------------------------------------------------------
 LUP for matrix a:
Matrix L:
[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.143 1.    0.    0.    0.   ]
 [1.    1.2   1.65  1.    0.    0.   ]
 [1.    1.231 2.09  1.96  1.    0.   ]
 [1.    1.25  2.406 2.76  2.333 1.   ]]
Matrix U:
[[ 3.     3.     3.     3.     3.     3.   ]
 [ 0.    -2.25  -1.125 -2.118 -1.636 -1.929]
 [ 0.     0.    -0.351 -0.063 -0.248 -0.144]
 [ 0.     0.     0.     0.01   0.024  0.02 ]
 [ 0.     0.     0.     0.     0.001  0.   ]
 [ 0.     0.     0.     0.     0.    -0.   ]]
Permutation matrix:
[[1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]]
Initial matrix:
[[3.    3.    3.    3.    3.    3.   ]
 [3.    1.875 1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 